### Cost of Living Index Data Extraction

In [2]:
# import modules
import pandas as pd
import numpy as np

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import URI
#from config import database

In [3]:
# parse in html tables and check for number of records per table
years = (2015, 2016, 2017, 2018, 2019)

all_COLI_data = []

for year in years:
    url = 'https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=' + str(year)
    print(url)
    COLI_data = pd.read_html(url)
    print(len(COLI_data[2]))
    all_COLI_data.append(COLI_data[2])


https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2015
119
https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2016
122
https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2017
121
https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2018
115
https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2019
119


### Cost of Living Index Data Transformation

In [4]:
# create a dataframe for each year
COLI_df_2015 = pd.DataFrame(all_COLI_data[0])
COLI_df_2016 = pd.DataFrame(all_COLI_data[1])
COLI_df_2017 = pd.DataFrame(all_COLI_data[2])
COLI_df_2018 = pd.DataFrame(all_COLI_data[3])
COLI_df_2019 = pd.DataFrame(all_COLI_data[4])

# add a column for year and assigning year value
COLI_df_2015["Year"] = [int(2015)]*len(all_COLI_data[0]) 
COLI_df_2016["Year"] = [int(2016)]*len(all_COLI_data[1]) 
COLI_df_2017["Year"] = [int(2017)]*len(all_COLI_data[2]) 
COLI_df_2018["Year"] = [int(2018)]*len(all_COLI_data[3]) 
COLI_df_2019["Year"] = [int(2019)]*len(all_COLI_data[4]) 

# add annual ranking based on row number + 1
COLI_df_2015["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2015)+1, step = 1)
COLI_df_2016["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2016)+1, step = 1)
COLI_df_2017["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2017)+1, step = 1)
COLI_df_2018["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2018)+1, step = 1)
COLI_df_2019["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2019)+1, step = 1)

# drop rank column
COLI_df_2015 = COLI_df_2015.drop(["Rank"], axis=1)
COLI_df_2016 = COLI_df_2016.drop(["Rank"], axis=1)
COLI_df_2017 = COLI_df_2017.drop(["Rank"], axis=1)
COLI_df_2018 = COLI_df_2018.drop(["Rank"], axis=1)
COLI_df_2019 = COLI_df_2019.drop(["Rank"], axis=1)

In [5]:
# concatenate dataframes for all years into a single dataframe
COLI_result_df = pd.concat([COLI_df_2015, COLI_df_2016, COLI_df_2017, 
                                COLI_df_2018, COLI_df_2019
                               ], 
                               axis=0, 
                               join="inner"
                              )
COLI_result_df

,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Year,Ranking
0,Switzerland,126.03,57.63,92.61,128.44,127.64,146.51,2015,1
1,Norway,118.59,48.70,84.43,110.06,141.45,96.30,2015,2
2,Venezuela,111.01,34.13,73.44,128.43,91.09,11.90,2015,3
3,Iceland,102.14,35.86,69.75,103.76,105.21,70.84,2015,4
4,Denmark,100.60,34.71,68.40,88.59,117.53,105.70,2015,5
...,...,...,...,...,...,...,...,...,...
114,Kosovo (Disputed Territory),26.18,8.05,17.48,23.21,19.57,61.82,2019,115
115,Venezuela,25.73,5.89,16.20,27.58,21.98,3.71,2019,116
116,India,24.17,5.84,15.37,24.20,17.15,66.91,2019,117
117,Tunisia,23.69,4.96,14.69,22.27,13.33,37.99,2019,118


In [6]:
# rename columns to remove spacing (required for SQL database)
COLI_result_df.rename(columns={"Cost of Living Index":"CostOfLiving_index",
                        "Rent Index":"Rent_index",
                        "Groceries Index":"Groceries_index",
                        "Restaurant Price Index":"RestaurantPrice_index",
                        "Local Purchasing Power Index":"LocalPurchPower_index"
                       }, 
               inplace=True
              )


In [7]:
# drop redundant columns
COLI_result_df.drop(["Cost of Living Plus Rent Index"], axis=1)

,Country,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index,Year,Ranking
0,Switzerland,126.03,57.63,128.44,127.64,146.51,2015,1
1,Norway,118.59,48.70,110.06,141.45,96.30,2015,2
2,Venezuela,111.01,34.13,128.43,91.09,11.90,2015,3
3,Iceland,102.14,35.86,103.76,105.21,70.84,2015,4
4,Denmark,100.60,34.71,88.59,117.53,105.70,2015,5
...,...,...,...,...,...,...,...,...
114,Kosovo (Disputed Territory),26.18,8.05,23.21,19.57,61.82,2019,115
115,Venezuela,25.73,5.89,27.58,21.98,3.71,2019,116
116,India,24.17,5.84,24.20,17.15,66.91,2019,117
117,Tunisia,23.69,4.96,22.27,13.33,37.99,2019,118


In [8]:
# resize dataframe (keeping relevant columns) and rearrange column order
COLI_result_df = pd.DataFrame(COLI_result_df[["Country", "Year", "Ranking", "CostOfLiving_index", "Rent_index", "Groceries_index", 
                   "RestaurantPrice_index","LocalPurchPower_index"
                  ]
                 ]
                             ) 
COLI_result_df


,Country,Year,Ranking,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index
0,Switzerland,2015,1,126.03,57.63,128.44,127.64,146.51
1,Norway,2015,2,118.59,48.70,110.06,141.45,96.30
2,Venezuela,2015,3,111.01,34.13,128.43,91.09,11.90
3,Iceland,2015,4,102.14,35.86,103.76,105.21,70.84
4,Denmark,2015,5,100.60,34.71,88.59,117.53,105.70
...,...,...,...,...,...,...,...,...
114,Kosovo (Disputed Territory),2019,115,26.18,8.05,23.21,19.57,61.82
115,Venezuela,2019,116,25.73,5.89,27.58,21.98,3.71
116,India,2019,117,24.17,5.84,24.20,17.15,66.91
117,Tunisia,2019,118,23.69,4.96,22.27,13.33,37.99


In [9]:
# count the number of years data for each country
country_df = pd.DataFrame(COLI_result_df.groupby(["Country"])["Year"].count())

# iterate over dataframe to drop countries where there are not 5 years of data
for index, row in country_df.iterrows():
    if row["Year"] !=5:
        country_df.drop(index, inplace = True)
        
country_df

,Year
Country,
Albania,5
Algeria,5
Argentina,5
Armenia,5
Australia,5
...,...
United States,5
Uruguay,5
Venezuela,5


In [10]:
# validate that all countries without 5 years of data have been dropped
check = country_df.loc[country_df["Year"] != 5]
check

,Year
Country,


In [11]:
# filter the database for countries with only 5 years of data
countries = list(country_df.index.values) 
countries


['Albania',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Bolivia',
 'Bosnia And Herzegovina',
 'Brazil',
 'Bulgaria',
 'Cambodia',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kuwait',
 'Latvia',
 'Lebanon',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mauritius',
 'Mexico',
 'Moldova',
 'Montenegro',
 'Morocco',
 'Nepal',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'North Macedonia',
 'Norway',
 'Oman',
 'Pakistan',
 'Palestine',
 'Panama',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',

In [12]:
# filter datafrome for countries with 5 years of data
COLI_df = pd.DataFrame(COLI_result_df[COLI_result_df["Country"].isin(countries)])
COLI_df

,Country,Year,Ranking,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index
0,Switzerland,2015,1,126.03,57.63,128.44,127.64,146.51
1,Norway,2015,2,118.59,48.70,110.06,141.45,96.30
2,Venezuela,2015,3,111.01,34.13,128.43,91.09,11.90
3,Iceland,2015,4,102.14,35.86,103.76,105.21,70.84
4,Denmark,2015,5,100.60,34.71,88.59,117.53,105.70
...,...,...,...,...,...,...,...,...
113,Egypt,2019,114,26.46,5.01,23.19,22.26,23.67
115,Venezuela,2019,116,25.73,5.89,27.58,21.98,3.71
116,India,2019,117,24.17,5.84,24.20,17.15,66.91
117,Tunisia,2019,118,23.69,4.96,22.27,13.33,37.99


In [13]:
# review simple stats as a reasonableness check (the "sniff" test)
COLI_df.describe()

,Year,Ranking,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index
count,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000
mean,2017.000000,59.766667,54.210815,20.376630,49.208926,46.782352,64.945407
std,1.415525,34.580567,20.090345,14.934197,20.344761,24.360431,33.209392
min,2015.000000,1.000000,20.400000,3.000000,18.250000,13.330000,3.350000
25%,2016.000000,29.000000,38.980000,10.282500,33.950000,29.025000,38.730000
50%,2017.000000,59.000000,50.145000,15.195000,44.315000,40.010000,54.955000
75%,2018.000000,90.000000,67.022500,27.025000,59.430000,59.705000,92.985000
max,2019.000000,122.000000,131.390000,84.620000,133.340000,141.450000,178.740000


In [14]:
# confirm same number of records per year for all years
check_df = pd.DataFrame(COLI_df.groupby(["Year"]).count())
check_df

,Country,Ranking,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index
Year,,,,,,,
2015,108,108,108,108,108,108,108
2016,108,108,108,108,108,108,108
2017,108,108,108,108,108,108,108
2018,108,108,108,108,108,108,108
2019,108,108,108,108,108,108,108


In [15]:
# parse in html tables and check for number of records per table
all_property_data = []

for year in years:
    url = 'https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=' + str(year)
    print(url)
    property_data = pd.read_html(url)
    print(len(property_data[2]))
    all_property_data.append(property_data[2])


https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=2015
121
https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=2016
102
https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=2017
99
https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=2018
89
https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=2019
93


In [16]:
# create a dataframe for each year
property_df_2015 = pd.DataFrame(all_property_data[0])
property_df_2016 = pd.DataFrame(all_property_data[1])
property_df_2017 = pd.DataFrame(all_property_data[2])
property_df_2018 = pd.DataFrame(all_property_data[3])
property_df_2019 = pd.DataFrame(all_property_data[4])

# add a column for year and assigning year value
property_df_2015["Year"] = [int(2015)]*len(all_property_data[0]) 
property_df_2016["Year"] = [int(2016)]*len(all_property_data[1]) 
property_df_2017["Year"] = [int(2017)]*len(all_property_data[2]) 
property_df_2018["Year"] = [int(2018)]*len(all_property_data[3]) 
property_df_2019["Year"] = [int(2019)]*len(all_property_data[4]) 

# drop rank column
property_df_2015 = property_df_2015.drop(["Rank"], axis=1)
property_df_2016 = property_df_2016.drop(["Rank"], axis=1)
property_df_2017 = property_df_2017.drop(["Rank"], axis=1)
property_df_2018 = property_df_2018.drop(["Rank"], axis=1)
property_df_2019 = property_df_2019.drop(["Rank"], axis=1)


In [17]:
# concatenate dataframes for all years to find common countries across all years
property_result_df = pd.concat([property_df_2015,property_df_2016, property_df_2017, 
                                property_df_2018, property_df_2019
                               ], 
                               axis=0, 
                               join="inner"
                              )
property_result_df

,Country,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,Price To Rent Ratio City Centre,Price To Rent Ratio Outside Of City Centre,Mortgage As A Percentage Of Income,Affordability Index,Year
0,Uganda,31.13,1.86,10.02,53.70,9.98,649.36,0.15,2015
1,Syria,30.49,3.95,3.15,25.33,31.75,283.67,0.35,2015
2,Hong Kong,30.09,2.67,2.77,37.49,36.08,192.99,0.52,2015
3,Myanmar,26.84,8.47,2.95,11.81,33.92,317.96,0.31,2015
4,Ghana,26.62,10.77,13.45,9.28,7.44,670.38,0.15,2015
...,...,...,...,...,...,...,...,...,...
88,Palestine,5.42,5.88,6.56,17.00,15.24,53.58,1.87,2019
89,United Arab Emirates,4.40,10.24,10.68,9.77,9.36,31.52,3.17,2019
90,South Africa,4.11,9.56,11.16,10.46,8.96,49.15,2.03,2019
91,United States,3.58,10.40,11.24,9.61,8.90,26.81,3.73,2019


In [18]:
# resize dataframe (keeping relevant columns) and rearrange column order
property_result_df = pd.DataFrame(property_result_df[["Country", "Year", "Price To Income Ratio", "Mortgage As A Percentage Of Income", 
                           "Affordability Index"
                          ]
                         ]
                          )
property_result_df

,Country,Year,Price To Income Ratio,Mortgage As A Percentage Of Income,Affordability Index
0,Uganda,2015,31.13,649.36,0.15
1,Syria,2015,30.49,283.67,0.35
2,Hong Kong,2015,30.09,192.99,0.52
3,Myanmar,2015,26.84,317.96,0.31
4,Ghana,2015,26.62,670.38,0.15
...,...,...,...,...,...
88,Palestine,2019,5.42,53.58,1.87
89,United Arab Emirates,2019,4.40,31.52,3.17
90,South Africa,2019,4.11,49.15,2.03
91,United States,2019,3.58,26.81,3.73


In [19]:
# rename columns to remove spacing (required for SQL database)
property_result_df.rename(columns={"Price To Income Ratio":"price_income_ratio",
                            "Mortgage As A Percentage Of Income":"mortage_income_ratio",
                            "Affordability Index":"affordability_index"
                           }, 
                   inplace=True
                  )
property_result_df

,Country,Year,price_income_ratio,mortage_income_ratio,affordability_index
0,Uganda,2015,31.13,649.36,0.15
1,Syria,2015,30.49,283.67,0.35
2,Hong Kong,2015,30.09,192.99,0.52
3,Myanmar,2015,26.84,317.96,0.31
4,Ghana,2015,26.62,670.38,0.15
...,...,...,...,...,...
88,Palestine,2019,5.42,53.58,1.87
89,United Arab Emirates,2019,4.40,31.52,3.17
90,South Africa,2019,4.11,49.15,2.03
91,United States,2019,3.58,26.81,3.73


In [20]:
# count the number of years data for each country
pcountry_df = pd.DataFrame(property_result_df.groupby(["Country"])["Year"].count())

# iterate over dataframe to drop countries where there are not 5 years of data
for index, row in pcountry_df.iterrows():
    if row["Year"] !=5:
        pcountry_df.drop(index, inplace = True)
        
pcountry_df

,Year
Country,
Albania,5
Algeria,5
Argentina,5
Armenia,5
Australia,5
...,...
United Arab Emirates,5
United Kingdom,5
United States,5


In [21]:
# validate that all countries without 5 years of data have been dropped
check = pcountry_df.loc[pcountry_df["Year"] != 5]
check

,Year
Country,


In [22]:
# filter the database for countries with only 5 years of data
final_countries = list(pcountry_df.index.values) 
final_countries

['Albania',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Bolivia',
 'Bosnia And Herzegovina',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'Estonia',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Latvia',
 'Lebanon',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mexico',
 'Moldova',
 'Montenegro',
 'Morocco',
 'Netherlands',
 'New Zealand',
 'North Macedonia',
 'Norway',
 'Pakistan',
 'Palestine',
 'Panama',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'Saudi Arabia',
 'Serbia',
 'Singapore',
 'Slovakia',
 'Slovenia',
 'South Africa',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerl

In [23]:
# filter property datafrome for countries with 5 years of data
property_df = pd.DataFrame(property_result_df[property_result_df["Country"].isin(final_countries)])
property_df

,Country,Year,price_income_ratio,mortage_income_ratio,affordability_index
2,Hong Kong,2015,30.09,192.99,0.52
5,China,2015,26.42,230.62,0.43
7,Taiwan,2015,24.97,152.28,0.66
8,Vietnam,2015,23.92,285.60,0.35
11,Singapore,2015,21.41,133.27,0.75
...,...,...,...,...,...
88,Palestine,2019,5.42,53.58,1.87
89,United Arab Emirates,2019,4.40,31.52,3.17
90,South Africa,2019,4.11,49.15,2.03
91,United States,2019,3.58,26.81,3.73


In [1]:
prop_list = property_df[["Country"]]
prop_list.to_csv("prop_list.csv")

NameError: name 'property_df' is not defined

In [23]:
# filter COLI datafrome for countries with 5 years of data in both dataframes
COLI_df = pd.DataFrame(COLI_result_df[COLI_result_df["Country"].isin(final_countries)])
COLI_df

,Country,Year,Ranking,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index
0,Switzerland,2015,1,126.03,57.63,128.44,127.64,146.51
1,Norway,2015,2,118.59,48.70,110.06,141.45,96.30
3,Iceland,2015,4,102.14,35.86,103.76,105.21,70.84
4,Denmark,2015,5,100.60,34.71,88.59,117.53,105.70
5,Australia,2015,6,99.32,49.47,94.31,90.63,110.39
...,...,...,...,...,...,...,...,...
111,Ukraine,2019,112,27.94,9.53,22.47,22.14,32.72
113,Egypt,2019,114,26.46,5.01,23.19,22.26,23.67
116,India,2019,117,24.17,5.84,24.20,17.15,66.91
117,Tunisia,2019,118,23.69,4.96,22.27,13.33,37.99


In [24]:
# review simple stats as a reasonableness check (the "sniff" test)
COLI_df.describe()

,Year,Ranking,CostOfLiving_index,Rent_index,Groceries_index,RestaurantPrice_index,LocalPurchPower_index
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,2017.000000,59.827273,54.782909,20.273250,49.117091,48.375386,68.259045
std,1.415823,36.155790,21.221499,14.943553,21.147333,25.833476,32.026640
min,2015.000000,1.000000,20.400000,4.500000,18.250000,13.330000,21.320000
25%,2016.000000,25.000000,38.232500,10.050000,33.055000,28.462500,41.147500
50%,2017.000000,60.000000,49.690000,15.215000,43.765000,41.170000,58.795000
75%,2018.000000,92.000000,70.192500,27.025000,61.530000,64.400000,95.427500
max,2019.000000,122.000000,131.390000,84.620000,133.340000,141.450000,178.740000


In [25]:
# review simple stats as a reasonableness check (the "sniff" test)
property_df.describe()

,Year,price_income_ratio,mortage_income_ratio,affordability_index
count,440.000000,440.000000,440.000000,440.000000
mean,2017.000000,12.242295,123.074455,1.218227
std,1.415823,5.786037,90.959945,0.794605
min,2015.000000,2.620000,19.430000,0.140000
25%,2016.000000,8.577500,62.147500,0.610000
50%,2017.000000,11.050000,93.655000,1.065000
75%,2018.000000,14.350000,163.245000,1.610000
max,2019.000000,49.420000,721.220000,5.150000


In [26]:
# create a connection to a Postgresql database
#rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')

In [27]:
#check for tables
#engine.table_names()

In [28]:
#use pandas to load csv converted DataFrame into database
#new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [29]:
#Confirm data has been added by querying the customer_name table (also checked in pgAdmin)
#pd.read_sql_query('select * from customer_name', con=engine).head()

In [30]:
# create csv files

#COLI_df.to_csv("COLI_raw.csv")
#property_df.to_csv("property_raw.csv")